<a href="https://colab.research.google.com/github/venugopals002/Venus/blob/main/BOPP_tape_SKU_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Install any missing packages
!pip install pandas matplotlib seaborn plotly openpyxl

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np

In [2]:
from google.colab import files

# Upload the Excel file
uploaded = files.upload()

# Load the Excel file into a DataFrame
df = pd.read_excel(list(uploaded.keys())[0])

# Display first few rows
df.head()

Saving BOPP tapes data 2024.xlsx to BOPP tapes data 2024.xlsx


,S no,Item No.,Item Description,Width in MM,Length (yard),Tapes per cartoon,Micron,UOM,Quantity,country,Tape Category,Tape Length (meters),Tape Thickness (microns),Units Sold in 2024 (No of tapes)
0,1,TCLT1001,Clear Tape 24mm x144roll,24,40.0,144,CL45,Ctn,199.0,Bahrain,BOPP tape,36.576,45,28656.0
1,2,TCLT1003,Clear Tape 36mm x48roll (1.5 inch),36,85.0,48,CL45,Ctn,227.0,Bahrain,BOPP tape,77.724,45,10896.0
2,3,TCLT1005,Clear Tape 72mm x 1000mtrs (3 inch),72,1000.0,4,CL50,Rolls,1380.0,Bahrain,BOPP tape,914.400,50,1380.0
3,4,TCLT1007,Clear Tape 4048 x 36roll - 2inch,48,85.0,36,CL40,Rolls,140652.0,Bahrain,BOPP tape,77.724,40,140652.0
4,5,TCLT1008,"""""Clear Tape 404872 x 72roll - 2""""""",48,40.0,72,CL40,Rolls,132264.0,Bahrain,BOPP tape,36.576,40,132264.0


In [4]:
# Step 3: Initial Data Exploration & Cleaning

# Check column names
print("Columns:\n", df.columns)

# Strip whitespace from column names for consistency
df.columns = df.columns.str.strip()

# Show basic info
print("\nBasic Info:")
df.info()

# Check for nulls
print("\nMissing Values:")
print(df.isnull().sum())

# Quick summary of numerical columns
print("\nStatistical Summary:")
print(df.describe())

Columns:
 Index(['S no', 'Item No.', 'Item Description', 'Width in MM', 'Length (yard)',
       'Tapes per cartoon', 'Micron', 'UOM', 'Quantity', 'country',
       'Tape Category', 'Tape Length (meters)', 'Tape Thickness (microns)',
       'Units Sold in 2024 (No of tapes)'],
      dtype='object')

Basic Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   S no                              116 non-null    int64  
 1   Item No.                          116 non-null    object 
 2   Item Description                  116 non-null    object 
 3   Width in MM                       116 non-null    int64  
 4   Length (yard)                     116 non-null    float64
 5   Tapes per cartoon                 116 non-null    int64  
 6   Micron                            114 non-null    object 
 7   UOM        

In [5]:
# Info and missing values
df.info()
print("\nMissing values:\n", df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   S no                              116 non-null    int64  
 1   Item No.                          116 non-null    object 
 2   Item Description                  116 non-null    object 
 3   Width in MM                       116 non-null    int64  
 4   Length (yard)                     116 non-null    float64
 5   Tapes per cartoon                 116 non-null    int64  
 6   Micron                            114 non-null    object 
 7   UOM                               116 non-null    object 
 8   Quantity                          116 non-null    float64
 9   country                           116 non-null    object 
 10  Tape Category                     116 non-null    object 
 11  Tape Length (meters)              116 non-null    float64
 12  Tape Thi

In [6]:
# Standardize numeric fields
df['Width in MM'] = pd.to_numeric(df['Width in MM'], errors='coerce')
df['Tape Length (meters)'] = pd.to_numeric(df['Tape Length (meters)'], errors='coerce')
df['Tape Thickness (microns)'] = pd.to_numeric(df['Tape Thickness (microns)'], errors='coerce')
df['Units Sold in 2024 (No of tapes)'] = pd.to_numeric(df['Units Sold in 2024 (No of tapes)'], errors='coerce')

# Drop rows with essential missing values
df.dropna(subset=['Width in MM', 'Tape Length (meters)', 'Tape Thickness (microns)', 'Units Sold in 2024 (No of tapes)'], inplace=True)

In [7]:
# Group by SKU (based on width, length, micron, country)
sku_summary = df.groupby(
    ['Width in MM', 'Tape Length (meters)', 'Tape Thickness (microns)', 'country']
)['Units Sold in 2024 (No of tapes)'].sum().reset_index()

sku_summary = sku_summary.sort_values(by='Units Sold in 2024 (No of tapes)', ascending=False)

# Add Rank
sku_summary['Rank'] = sku_summary['Units Sold in 2024 (No of tapes)'].rank(ascending=False).astype(int)

sku_summary.head(10)

,Width in MM,Tape Length (meters),Tape Thickness (microns),country,Units Sold in 2024 (No of tapes),Rank
43,48,86.868,50,KSA,1298701.185,1
41,48,86.868,45,KSA,511768.986,2
40,48,86.868,40,KSA,418714.998,3
47,48,91.440,50,KSA,296970.000,4
39,48,82.296,50,KSA,232953.335,5
32,48,77.724,40,KSA,223309.788,6
35,48,77.724,45,KSA,183980.001,7
6,24,77.724,45,KSA,181740.072,8
25,48,64.008,50,KSA,156480.000,9
31,48,77.724,40,Bahrain,140652.000,10


from matplotlib import pyplot as plt
_df_0['Width in MM'].plot(kind='hist', bins=20, title='Width in MM')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Tape Length (meters)'].plot(kind='hist', bins=20, title='Tape Length (meters)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Tape Thickness (microns)'].plot(kind='hist', bins=20, title='Tape Thickness (microns)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Units Sold in 2024 (No of tapes)'].plot(kind='hist', bins=20, title='Units Sold in 2024 (No of tapes)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='Tape Length (meters)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='Tape Thickness (microns)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='Units Sold in 2024 (No of tapes)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [10]:
# Calculate total sales
total_units = sku_summary['Units Sold in 2024 (No of tapes)'].sum()

# Add % share column
sku_summary['% of Total Sales'] = (sku_summary['Units Sold in 2024 (No of tapes)'] / total_units * 100).round(2)

# Suggest to eliminate if SKU has < 1% sales share
sku_summary['Recommendation'] = np.where(sku_summary['% of Total Sales'] < 1,
                                         'Consider Eliminating or Merging',
                                         'Retain')

sku_summary.head(20)

,Width in MM,Tape Length (meters),Tape Thickness (microns),country,Units Sold in 2024 (No of tapes),Rank,% of Total Sales,Recommendation
43,48,86.8680,50,KSA,1298701.185,1,23.59,Retain
41,48,86.8680,45,KSA,511768.986,2,9.30,Retain
40,48,86.8680,40,KSA,418714.998,3,7.61,Retain
47,48,91.4400,50,KSA,296970.000,4,5.39,Retain
39,48,82.2960,50,KSA,232953.335,5,4.23,Retain
32,48,77.7240,40,KSA,223309.788,6,4.06,Retain
35,48,77.7240,45,KSA,183980.001,7,3.34,Retain
6,24,77.7240,45,KSA,181740.072,8,3.30,Retain
25,48,64.0080,50,KSA,156480.000,9,2.84,Retain
31,48,77.7240,40,Bahrain,140652.000,10,2.55,Retain


from matplotlib import pyplot as plt
_df_21['Width in MM'].plot(kind='hist', bins=20, title='Width in MM')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_22['Tape Length (meters)'].plot(kind='hist', bins=20, title='Tape Length (meters)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_23['Tape Thickness (microns)'].plot(kind='hist', bins=20, title='Tape Thickness (microns)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_24['Units Sold in 2024 (No of tapes)'].plot(kind='hist', bins=20, title='Units Sold in 2024 (No of tapes)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_39['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_39, x='Tape Length (meters)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_40['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_40, x='Tape Thickness (microns)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_41['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_41, x='Units Sold in 2024 (No of tapes)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [11]:
top15 = sku_summary.sort_values(by='Units Sold in 2024 (No of tapes)', ascending=False).head(15)

fig = px.bar(
    top15,
    x='Units Sold in 2024 (No of tapes)',
    y=top15.apply(lambda x: f"{x['Width in MM']}mm x {x['Tape Length (meters)']}m x {x['Tape Thickness (microns)']}μ ({x['country']})", axis=1),
    orientation='h',
    title='Top 15 BOPP Tape SKUs by Units Sold (2024)',
    labels={'y': 'SKU (Width x Length x Thickness)'},
    height=600
)
fig.show()

In [12]:
fig = px.pie(
    sku_summary.head(10),
    names=sku_summary.head(10).apply(lambda x: f"{x['Width in MM']}mm x {x['Tape Length (meters)']}m x {x['Tape Thickness (microns)']}μ ({x['country']})", axis=1),
    values='Units Sold in 2024 (No of tapes)',
    title='Top 10 SKU Contribution to Total Sales',
    hole=0.4
)
fig.show()

In [13]:
# Filter rows that are recommended to be eliminated
to_eliminate = sku_summary[sku_summary['Recommendation'] == 'Consider Eliminating or Merging']

print("🔻 SKUs Recommended for Elimination or Merging:")
to_eliminate[['Width in MM', 'Tape Length (meters)', 'Tape Thickness (microns)', 'country',
              'Units Sold in 2024 (No of tapes)', '% of Total Sales']]

🔻 SKUs Recommended for Elimination or Merging:


,Width in MM,Tape Length (meters),Tape Thickness (microns),country,Units Sold in 2024 (No of tapes),% of Total Sales
46,48,91.44000,45,KSA,53239.9968,0.97
63,48,868.68000,50,KSA,49530.1662,0.90
23,48,64.00800,40,KSA,46920.0000,0.85
56,48,182.88000,40,KSA,44952.0000,0.82
13,36,82.29600,40,KSA,40760.0000,0.74
68,48,914.40000,50,KSA,38529.9996,0.70
5,24,77.72400,45,Bahrain,33768.0000,0.61
33,48,77.72400,42,Bahrain,31788.0000,0.58
9,24,91.44000,45,Bahrain,31608.0000,0.57
4,24,36.57600,45,Bahrain,28656.0000,0.52


from matplotlib import pyplot as plt
_df_42['Width in MM'].plot(kind='hist', bins=20, title='Width in MM')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_43['Tape Length (meters)'].plot(kind='hist', bins=20, title='Tape Length (meters)')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_44['Tape Thickness (microns)'].plot(kind='hist', bins=20, title='Tape Thickness (microns)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_56['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_56, x='Tape Length (meters)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_57['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_57, x='Tape Thickness (microns)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning:



Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.




from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_58['country'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_58, x='Units Sold in 2024 (No of tapes)', y='country', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [14]:
sku_summary.to_excel("SKU_Optimization_Report.xlsx", index=False)
files.download("SKU_Optimization_Report.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>